In [1]:
# !pip install -r requirements.txt

In [2]:
from llm import chat_openai
import numpy as np 
import json
import logging 
import matplotlib.pyplot as plt
import os
import openai
import re
import subprocess
from pathlib import Path
import shutil
import time 

from utils.misc import * 
# from utils.file_utils import find_files_with_substring
# from utils.create_task import create_task
from utils.extract_task_code import *

In [3]:
# print(chat_openai("hi")[0])

In [4]:
# Starting based on eureka.py main
EUREKA_ROOT_DIR = os.getcwd()
ISAAC_ROOT_DIR = f"{EUREKA_ROOT_DIR}/../isaacgymenvs/isaacgymenvs"

task = "Humanoid"
task_description = "to make the humanoid run as fast as possible"
suffix = "GPT"
model = "gpt-3.5-turbo"
print(f"Using LLM: {model}")
print("Task: " + task)
print("Task description: " + task_description)

Using LLM: gpt-3.5-turbo
Task: Humanoid
Task description: to make the humanoid run as fast as possible


In [5]:
env_name = "humanoid"
env_parent = 'isaac' if f'{env_name}.py' in os.listdir(f'{EUREKA_ROOT_DIR}/envs/isaac') else 'dexterity'
task_file = f'{EUREKA_ROOT_DIR}/envs/{env_parent}/{env_name}.py'
task_obs_file = f'{EUREKA_ROOT_DIR}/envs/{env_parent}/{env_name}_obs.py'
shutil.copy(task_obs_file, f"env_init_obs.py")
task_code_string  = file_to_string(task_file)
task_obs_code_string  = file_to_string(task_obs_file)
output_file = f"{ISAAC_ROOT_DIR}/tasks/{env_name}{suffix.lower()}.py"

In [6]:
# Loading all text prompts
prompt_dir = f'{EUREKA_ROOT_DIR}/utils/prompts'
initial_system = file_to_string(f'{prompt_dir}/initial_system.txt')
code_output_tip = file_to_string(f'{prompt_dir}/code_output_tip.txt')
code_feedback = file_to_string(f'{prompt_dir}/code_feedback.txt')
initial_user = file_to_string(f'{prompt_dir}/initial_user.txt')
reward_signature = file_to_string(f'{prompt_dir}/reward_signature.txt')
policy_feedback = file_to_string(f'{prompt_dir}/policy_feedback.txt')
execution_error_feedback = file_to_string(f'{prompt_dir}/execution_error_feedback.txt')

In [7]:
initial_system = initial_system.format(task_reward_signature_string=reward_signature) + code_output_tip
initial_user = initial_user.format(task_obs_code_string=task_obs_code_string, task_description=task_description)
messages = [{"role": "system", "content": initial_system}, {"role": "user", "content": initial_user}]

task_code_string = task_code_string.replace(task, task+suffix)

In [8]:
DUMMY_FAILURE = -10000.
max_successes = []
max_successes_reward_correlation = []
execute_rates = []
best_code_paths = []
max_success_overall = DUMMY_FAILURE
max_success_reward_correlation_overall = DUMMY_FAILURE
max_reward_code_path = None 
num_iterations = 1

In [13]:
# Eureka generation loop
for iter in range(num_iterations):
    # Get Eureka response
    responses = []
    response_cur = None
    total_samples = 0
    total_token = 0
    total_completion_token = 0
    # chunk_size = 3 if "gpt-3.5" in model else 4
    chunk_size = 1 if "gpt-3.5" in model else 4

    print(f"Iteration {iter}: Generating {chunk_size} samples with {model}")

    while True:
        if total_samples >= chunk_size:
            break
        print("Messages: ", messages)
# Messages:  Messages:  [{'role': 'system', 'content': 'You are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effective as possible.\nYour goal is to write a reward function for the environment that will help the agent learn the task described in text. \nYour reward function should use useful variables from the environment as inputs. As an example,\nthe reward function signature can be: @torch.jit.script\ndef compute_reward(object_pos: torch.Tensor, goal_pos: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:\n    ...\n    return reward, {}\n\nSince the reward function will be decorated with @torch.jit.script,\nplease make sure that the code is compatible with TorchScript (e.g., use torch tensor instead of numpy array). \nMake sure any new tensor or variable you introduce is on the same device as the input tensors. The output of the reward function should consist of two items:\n    (1) the total reward,\n    (2) a dictionary of each individual reward component.\nThe code output should be formatted as a python code string: "```python ... ```".\n\nSome helpful tips for writing the reward function code:\n    (1) You may find it helpful to normalize the reward to a fixed range by applying transformations like torch.exp to the overall reward or its components\n    (2) If you choose to transform a reward component, then you must also introduce a temperature parameter inside the transformation function; this parameter must be a named variable in the reward function and it must not be an input variable. Each transformed reward component should have its own temperature variable\n    (3) Make sure the type of each input variable is correctly specified; a float input variable should not be specified as torch.Tensor\n    (4) Most importantly, the reward code\'s input variables must contain only attributes of the provided environment class definition (namely, variables that have prefix self.). Under no circumstance can you introduce new input variables.\n'}, {'role': 'user', 'content': 'The Python environment is class Humanoid(VecTask):\n    """Rest of the environment definition omitted."""\n    def compute_observations(self):\n        self.gym.refresh_dof_state_tensor(self.sim)\n        self.gym.refresh_actor_root_state_tensor(self.sim)\n\n        self.gym.refresh_force_sensor_tensor(self.sim)\n        self.gym.refresh_dof_force_tensor(self.sim)\n        self.obs_buf[:], self.potentials[:], self.prev_potentials[:], self.up_vec[:], self.heading_vec[:] = compute_humanoid_observations(\n            self.obs_buf, self.root_states, self.targets, self.potentials,\n            self.inv_start_rot, self.dof_pos, self.dof_vel, self.dof_force_tensor,\n            self.dof_limits_lower, self.dof_limits_upper, self.dof_vel_scale,\n            self.vec_sensor_tensor, self.actions, self.dt, self.contact_force_scale, self.angular_velocity_scale,\n            self.basis_vec0, self.basis_vec1)\n\ndef compute_humanoid_observations(obs_buf, root_states, targets, potentials, inv_start_rot, dof_pos, dof_vel,\n                                  dof_force, dof_limits_lower, dof_limits_upper, dof_vel_scale,\n                                  sensor_force_torques, actions, dt, contact_force_scale, angular_velocity_scale,\n                                  basis_vec0, basis_vec1):\n    # type: (Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, float, Tensor, Tensor, float, float, float, Tensor, Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor]\n\n    torso_position = root_states[:, 0:3]\n    torso_rotation = root_states[:, 3:7]\n    velocity = root_states[:, 7:10]\n    ang_velocity = root_states[:, 10:13]\n\n    to_target = targets - torso_position\n    to_target[:, 2] = 0\n\n    prev_potentials_new = potentials.clone()\n    potentials = -torch.norm(to_target, p=2, dim=-1) / dt\n\n    torso_quat, up_proj, heading_proj, up_vec, heading_vec = compute_heading_and_up(\n        torso_rotation, inv_start_rot, to_target, basis_vec0, basis_vec1, 2)\n\n    vel_loc, angvel_loc, roll, pitch, yaw, angle_to_target = compute_rot(\n        torso_quat, velocity, ang_velocity, targets, torso_position)\n\n    roll = normalize_angle(roll).unsqueeze(-1)\n    yaw = normalize_angle(yaw).unsqueeze(-1)\n    angle_to_target = normalize_angle(angle_to_target).unsqueeze(-1)\n    dof_pos_scaled = unscale(dof_pos, dof_limits_lower, dof_limits_upper)\n\n    obs = torch.cat((torso_position[:, 2].view(-1, 1), vel_loc, angvel_loc * angular_velocity_scale,\n                     yaw, roll, angle_to_target, up_proj.unsqueeze(-1), heading_proj.unsqueeze(-1),\n                     dof_pos_scaled, dof_vel * dof_vel_scale, dof_force * contact_force_scale,\n                     sensor_force_torques.view(-1, 12) * contact_force_scale, actions), dim=-1)\n\n    return obs, potentials, prev_potentials_new, up_vec, heading_vec\n\n\n\n. Write a reward function for the following task: to make the humanoid run as fast as possible.'}]
        for attempt in range(1000):
            try:
                response_cur = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    temperature=1.0,
                    n=chunk_size
                )
                total_samples += chunk_size
                break
            except Exception as e:
                if attempt >= 10:
                    chunk_size = max(int(chunk_size / 2), 1)
                    print("Current Chunk Size", chunk_size)
                print(f"Attempt {attempt+1} failed with error: {e}")
                time.sleep(1)
        if response_cur is None:
            print("Code terminated due to too many failed attempts!")
            exit()

        responses.extend(response_cur["choices"])
        prompt_tokens = response_cur["usage"]["prompt_tokens"]
        total_completion_token += response_cur["usage"]["completion_tokens"]
        total_token += response_cur["usage"]["total_tokens"]

    if chunk_size == 1:
        print(f"Iteration {iter}: GPT Output:\n " + responses[0]["message"]["content"] + "\n")

    # Logging Token Information
    print(f"Iteration {iter}: Prompt Tokens: {prompt_tokens}, Completion Tokens: {total_completion_token}, Total Tokens: {total_token}")
    print("RESPONSES: ", responses)

    code_runs = [] 
    rl_runs = []
    for response_id in range(chunk_size):
        response_cur = responses[response_id]["message"]["content"]
        print(f"Iteration {iter}: Processing Code Run {response_id}")

        # SIMULATION STEP? I think
        # TODO: replace with having a LLM evaluate this step and act as a simulator
        
        # TODO: update messages to include response_cur which is the proposed model and add a query for an eval step.

        print("before simulation messages: ", messages)

        sim_instructions = '''Assume that you are about to start testing. Theoretically simulate and predict the most likely outcome. Think step by step at the deepest and most informative level. What will happen? You are to predict the result.'''
        sim_messages = messages + [
            {"role": "assistant", "content": response_cur}, 
            {"role": "user", "content": sim_instructions}
        ]
        print("simulation messages: ", sim_messages)

        response_sim = openai.ChatCompletion.create(
                model=model,
                messages=sim_messages,
                temperature=0.0,
                n=chunk_size
            )
        code_runs.append(response_cur)
        rl_runs.append(response_sim)
        print("...processed code run...", response_sim)

    # Gather RL training results and construct reward reflection as prompt for next LLM
    print("---Gather RL training results and construct reward reflection as prompt for next LLM---")
    code_feedbacks = []
    contents = []
    successes = []
    reward_correlations = []
    code_paths = []

    exec_success = False 
    for response_id, (code_run, rl_run) in enumerate(zip(code_runs, rl_runs)):
        print("\nKEYS, VALUES: ", response_id, (code_run, rl_run))
        # rl_run.communicate()
        # rl_filepath = f"env_iter{iter}_response{response_id}.txt"
        code_paths.append(f"env_iter{iter}_response{response_id}.py")
        try:
            # with open(rl_filepath, 'r') as f:
            #     stdout_str = f.read() 
            stdout_str = rl_run
        except: 
            content = execution_error_feedback.format(traceback_msg="Code Run cannot be executed due to function signature error! Please re-write an entirely new reward function!")
            content += code_output_tip
            contents.append(content) 
            successes.append(DUMMY_FAILURE)
            reward_correlations.append(DUMMY_FAILURE)
            continue

        content = ''
        traceback_msg = filter_traceback(stdout_str)

        if traceback_msg == '':
            # TODO: FIX THIS

            # If RL execution has no error, provide policy statistics feedback
            exec_success = True
            content += policy_feedback
            
            # # Compute Correlation between Human-Engineered and GPT Rewards
            # if "gt_reward" in tensorboard_logs and "gpt_reward" in tensorboard_logs:
            #     gt_reward = np.array(tensorboard_logs["gt_reward"])
            #     gpt_reward = np.array(tensorboard_logs["gpt_reward"])
            #     reward_correlation = np.corrcoef(gt_reward, gpt_reward)[0, 1]
            #     reward_correlations.append(reward_correlation)

            # # Add reward components log to the feedback
            # for metric in tensorboard_logs:
            #     if "/" not in metric:
            #         metric_cur = ['{:.2f}'.format(x) for x in tensorboard_logs[metric][::epoch_freq]]
            #         metric_cur_max = max(tensorboard_logs[metric])
            #         metric_cur_mean = sum(tensorboard_logs[metric]) / len(tensorboard_logs[metric])
            #         if "consecutive_successes" == metric:
            #             successes.append(metric_cur_max)
            #         metric_cur_min = min(tensorboard_logs[metric])
            #         if metric != "gt_reward" and metric != "gpt_reward":
            #             if metric != "consecutive_successes":
            #                 metric_name = metric 
            #             else:
            #                 metric_name = "task_score"
            #             content += f"{metric_name}: {metric_cur}, Max: {metric_cur_max:.2f}, Mean: {metric_cur_mean:.2f}, Min: {metric_cur_min:.2f} \n"                    
            #         else:
            #             # Provide ground-truth score when success rate not applicable
            #             if "consecutive_successes" not in tensorboard_logs:
            #                 content += f"ground-truth score: {metric_cur}, Max: {metric_cur_max:.2f}, Mean: {metric_cur_mean:.2f}, Min: {metric_cur_min:.2f} \n"  
            successes.append(10) # TODO: hard coded for now
            reward_correlations.append(10) # TODO: hard coded for now
            code_feedback = stdout_str # TODO: hard coded for now               
            code_feedbacks.append(code_feedback)
            content += code_feedback  
        else:
            # Otherwise, provide execution traceback error feedback
            successes.append(DUMMY_FAILURE)
            reward_correlations.append(DUMMY_FAILURE)
            content += execution_error_feedback.format(traceback_msg=traceback_msg)

        content += code_output_tip
        contents.append(content) 

    print("contents: ", contents)

    # Repeat the iteration if all code generation failed
    if not exec_success and chunk_size != 1:
        execute_rates.append(0.)
        max_successes.append(DUMMY_FAILURE)
        max_successes_reward_correlation.append(DUMMY_FAILURE)
        best_code_paths.append(None)
        print("All code generation failed! Repeat this iteration from the current message checkpoint!")
        continue

    print("\nFINISHED ROUND")
    print("successes", successes)
    print("contents", contents)


    # Select the best code sample based on the success rate
    best_sample_idx = np.argmax(np.array(successes))
    best_content = contents[best_sample_idx]

    max_success = successes[best_sample_idx]
    max_success_reward_correlation = reward_correlations[best_sample_idx]
    execute_rate = np.sum(np.array(successes) >= 0.) / chunk_size

    # Update the best Eureka Output
    if max_success > max_success_overall:
        max_success_overall = max_success
        max_success_reward_correlation_overall = max_success_reward_correlation
        max_reward_code_path = code_paths[best_sample_idx]

    execute_rates.append(execute_rate)
    max_successes.append(max_success)
    max_successes_reward_correlation.append(max_success_reward_correlation)
    best_code_paths.append(code_paths[best_sample_idx])

    print(f"Iteration {iter}: Max Success: {max_success}, Execute Rate: {execute_rate}, Max Success Reward Correlation: {max_success_reward_correlation}")
    print(f"Iteration {iter}: Best Generation ID: {best_sample_idx}")
    print(f"Iteration {iter}: GPT Output Content:\n" +  responses[best_sample_idx]["message"]["content"] + "\n")
    print(f"Iteration {iter}: User Content:\n" + best_content + "\n")
        
    # Plot the success rate
    fig, axs = plt.subplots(2, figsize=(6, 6))
    fig.suptitle(task)

    x_axis = np.arange(len(max_successes))

    axs[0].plot(x_axis, np.array(max_successes))
    axs[0].set_title("Max Success")
    axs[0].set_xlabel("Iteration")

    axs[1].plot(x_axis, np.array(execute_rates))
    axs[1].set_title("Execute Rate")
    axs[1].set_xlabel("Iteration")

    fig.tight_layout(pad=3.0)
    plt.savefig('summary.png')
    np.savez('summary.npz', max_successes=max_successes, execute_rates=execute_rates, best_code_paths=best_code_paths, max_successes_reward_correlation=max_successes_reward_correlation)

    if len(messages) == 2:
        messages += [{"role": "assistant", "content": responses[best_sample_idx]["message"]["content"]}]
        messages += [{"role": "user", "content": best_content}]
    else:
        assert len(messages) == 4
        messages[-2] = {"role": "assistant", "content": responses[best_sample_idx]["message"]["content"]}
        messages[-1] = {"role": "user", "content": best_content}

    # Save dictionary as JSON file
    with open('messages.json', 'w') as file:
        json.dump(messages, file, indent=4)

    print("\n***SAVED AND FINISHED ONE ROUND ***\n", "messages: ", messages)

Iteration 0: Generating 1 samples with gpt-3.5-turbo
Messages:  [{'role': 'system', 'content': 'You are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effective as possible.\nYour goal is to write a reward function for the environment that will help the agent learn the task described in text. \nYour reward function should use useful variables from the environment as inputs. As an example,\nthe reward function signature can be: @torch.jit.script\ndef compute_reward(object_pos: torch.Tensor, goal_pos: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:\n    ...\n    return reward, {}\n\nSince the reward function will be decorated with @torch.jit.script,\nplease make sure that the code is compatible with TorchScript (e.g., use torch tensor instead of numpy array). \nMake sure any new tensor or variable you introduce is on the same device as the input tensors. The output of the reward function should consist of two items:\n    (1) the

Iteration 0: GPT Output:
 Based on the task description, a possible reward function for making the humanoid run as fast as possible could be to provide a positive reward based on the change in velocity of the humanoid. The reward can be calculated by comparing the current velocity with the previous velocity.

Here is the reward function implementation:

```python
import torch
from typing import Tuple, Dict

@torch.jit.script
def compute_reward(velocity: torch.Tensor, prev_velocity: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:
    velocity_diff = velocity - prev_velocity
    reward = torch.norm(velocity_diff, p=2)

    reward_components = {
        "velocity_reward": reward
    }

    return reward, reward_components
```

Explanation:
- The function takes in the current velocity of the humanoid (`velocity`) and the previous velocity (`prev_velocity`) as input.
- `velocity_diff` calculates the difference between the current velocity and the previous velocity of the huma

AttributeError: split

: 

In [10]:
# # Evaluate the best reward code many times
# if max_reward_code_path is None: 
#     print("All iterations of code generation failed, aborting...")
#     print("Please double check the output env_iter*_response*.txt files for repeating errors!")
#     exit()
# print(f"Task: {task}, Max Training Success {max_success_overall}, Correlation {max_success_reward_correlation_overall}, Best Reward Code Path: {max_reward_code_path}")
# print(f"Evaluating best reward code {cfg.num_eval} times")
# shutil.copy(max_reward_code_path, output_file)

# eval_runs = []
# for i in range(cfg.num_eval):
#     set_freest_gpu()
    
#     # Execute the python file with flags
#     rl_filepath = f"reward_code_eval{i}.txt"
#     with open(rl_filepath, 'w') as f:
#         process = subprocess.Popen(['python', '-u', f'{ISAAC_ROOT_DIR}/train.py',  
#                                     'hydra/output=subprocess',
#                                     f'task={task}{suffix}', f'wandb_activate={cfg.use_wandb}',
#                                     f'wandb_entity={cfg.wandb_username}', f'wandb_project={cfg.wandb_project}',
#                                     f'headless={not cfg.capture_video}', f'capture_video={cfg.capture_video}', 'force_render=False', f'seed={i}',
#                                     ],
#                                     stdout=f, stderr=f)

#     block_until_training(rl_filepath)
#     eval_runs.append(process)

# reward_code_final_successes = []
# reward_code_correlations_final = []
# for i, rl_run in enumerate(eval_runs):
#     rl_run.communicate()
#     rl_filepath = f"reward_code_eval{i}.txt"
#     with open(rl_filepath, 'r') as f:
#         stdout_str = f.read() 
#     lines = stdout_str.split('\n')
#     for i, line in enumerate(lines):
#         if line.startswith('Tensorboard Directory:'):
#             break 
#     tensorboard_logdir = line.split(':')[-1].strip() 
#     tensorboard_logs = load_tensorboard_logs(tensorboard_logdir)
#     max_success = max(tensorboard_logs['consecutive_successes'])
#     reward_code_final_successes.append(max_success)

#     if "gt_reward" in tensorboard_logs and "gpt_reward" in tensorboard_logs:
#         gt_reward = np.array(tensorboard_logs["gt_reward"])
#         gpt_reward = np.array(tensorboard_logs["gpt_reward"])
#         reward_correlation = np.corrcoef(gt_reward, gpt_reward)[0, 1]
#         reward_code_correlations_final.append(reward_correlation)

# print(f"Final Success Mean: {np.mean(reward_code_final_successes)}, Std: {np.std(reward_code_final_successes)}, Raw: {reward_code_final_successes}")
# print(f"Final Correlation Mean: {np.mean(reward_code_correlations_final)}, Std: {np.std(reward_code_correlations_final)}, Raw: {reward_code_correlations_final}")
# np.savez('final_eval.npz', reward_code_final_successes=reward_code_final_successes, reward_code_correlations_final=reward_code_correlations_final)

In [11]:
print('''
      messages:  [{'role': 'system', 'content': 'You are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effective as possible.\nYour goal is to write a reward function for the environment that will help the agent learn the task described in text. \nYour reward function should use useful variables from the environment as inputs. As an example,\nthe reward function signature can be: @torch.jit.script\ndef compute_reward(object_pos: torch.Tensor, goal_pos: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:\n    ...\n    return reward, {}\n\nSince the reward function will be decorated with @torch.jit.script,\nplease make sure that the code is compatible with TorchScript (e.g., use torch tensor instead of numpy array). \nMake sure any new tensor or variable you introduce is on the same device as the input tensors. The output of the reward function should consist of two items:\n    (1) the total reward,\n    (2) a dictionary of each individual reward component.\nThe code output should be formatted as a python code string: "```python ... ```".\n\nSome helpful tips for writing the reward function code:\n    (1) You may find it helpful to normalize the reward to a fixed range by applying transformations like torch.exp to the overall reward or its components\n    (2) If you choose to transform a reward component, then you must also introduce a temperature parameter inside the transformation function; this parameter must be a named variable in the reward function and it must not be an input variable. Each transformed reward component should have its own temperature variable\n    (3) Make sure the type of each input variable is correctly specified; a float input variable should not be specified as torch.Tensor\n    (4) Most importantly, the reward code\'s input variables must contain only attributes of the provided environment class definition (namely, variables that have prefix self.). Under no circumstance can you introduce new input variables.\n'}, {'role': 'user', 'content': 'The Python environment is class Humanoid(VecTask):\n    """Rest of the environment definition omitted."""\n    def compute_observations(self):\n        self.gym.refresh_dof_state_tensor(self.sim)\n        self.gym.refresh_actor_root_state_tensor(self.sim)\n\n        self.gym.refresh_force_sensor_tensor(self.sim)\n        self.gym.refresh_dof_force_tensor(self.sim)\n        self.obs_buf[:], self.potentials[:], self.prev_potentials[:], self.up_vec[:], self.heading_vec[:] = compute_humanoid_observations(\n            self.obs_buf, self.root_states, self.targets, self.potentials,\n            self.inv_start_rot, self.dof_pos, self.dof_vel, self.dof_force_tensor,\n            self.dof_limits_lower, self.dof_limits_upper, self.dof_vel_scale,\n            self.vec_sensor_tensor, self.actions, self.dt, self.contact_force_scale, self.angular_velocity_scale,\n            self.basis_vec0, self.basis_vec1)\n\ndef compute_humanoid_observations(obs_buf, root_states, targets, potentials, inv_start_rot, dof_pos, dof_vel,\n                                  dof_force, dof_limits_lower, dof_limits_upper, dof_vel_scale,\n                                  sensor_force_torques, actions, dt, contact_force_scale, angular_velocity_scale,\n                                  basis_vec0, basis_vec1):\n    # type: (Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, float, Tensor, Tensor, float, float, float, Tensor, Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor]\n\n    torso_position = root_states[:, 0:3]\n    torso_rotation = root_states[:, 3:7]\n    velocity = root_states[:, 7:10]\n    ang_velocity = root_states[:, 10:13]\n\n    to_target = targets - torso_position\n    to_target[:, 2] = 0\n\n    prev_potentials_new = potentials.clone()\n    potentials = -torch.norm(to_target, p=2, dim=-1) / dt\n\n    torso_quat, up_proj, heading_proj, up_vec, heading_vec = compute_heading_and_up(\n        torso_rotation, inv_start_rot, to_target, basis_vec0, basis_vec1, 2)\n\n    vel_loc, angvel_loc, roll, pitch, yaw, angle_to_target = compute_rot(\n        torso_quat, velocity, ang_velocity, targets, torso_position)\n\n    roll = normalize_angle(roll).unsqueeze(-1)\n    yaw = normalize_angle(yaw).unsqueeze(-1)\n    angle_to_target = normalize_angle(angle_to_target).unsqueeze(-1)\n    dof_pos_scaled = unscale(dof_pos, dof_limits_lower, dof_limits_upper)\n\n    obs = torch.cat((torso_position[:, 2].view(-1, 1), vel_loc, angvel_loc * angular_velocity_scale,\n                     yaw, roll, angle_to_target, up_proj.unsqueeze(-1), heading_proj.unsqueeze(-1),\n                     dof_pos_scaled, dof_vel * dof_vel_scale, dof_force * contact_force_scale,\n                     sensor_force_torques.view(-1, 12) * contact_force_scale, actions), dim=-1)\n\n    return obs, potentials, prev_potentials_new, up_vec, heading_vec\n\n\n\n. Write a reward function for the following task: to make the humanoid run as fast as possible.'}, {'role': 'assistant', 'content': "Apologies for the confusion. Here's the updated reward function code that follows the specified format:\n\n```python\n@torch.jit.script\ndef compute_reward(velocity: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:\n    temperature = 1.0\n\n    velocity_reward = torch.exp(temperature * velocity.norm(dim=-1))  # Apply temperature and exponential transformation\n\n    reward_components = {\n        'velocity_reward': velocity_reward\n    }\n\n    total_reward = torch.sum(velocity_reward)  # Compute the total reward as the sum of individual rewards\n\n    return total_reward, reward_components\n```\n\nPlease note the following updates:\n- A temperature parameter `temperature` has been introduced to control the transformation of the reward component. You can adjust its value to change the emphasis on higher velocities.\n- The individual reward component `velocity_reward` is calculated using the temperature and exponential transformation on the norm of the velocity.\n- The total reward is computed using `torch.sum`.\n- The output is now a tuple consisting of the total reward and the dictionary of individual reward components.\n\nYou can use this updated reward function to train the humanoid to run as fast as possible.\n"}, {'role': 'user', 'content': 'Great results!The output of the reward function should consist of two items:\n    (1) the total reward,\n    (2) a dictionary of each individual reward component.\nThe code output should be formatted as a python code string: "```python ... ```".\n\nSome helpful tips for writing the reward function code:\n    (1) You may find it helpful to normalize the reward to a fixed range by applying transformations like torch.exp to the overall reward or its components\n    (2) If you choose to transform a reward component, then you must also introduce a temperature parameter inside the transformation function; this parameter must be a named variable in the reward function and it must not be an input variable. Each transformed reward component should have its own temperature variable\n    (3) Make sure the type of each input variable is correctly specified; a float input variable should not be specified as torch.Tensor\n    (4) Most importantly, the reward code\'s input variables must contain only attributes of the provided environment class definition (namely, variables that have prefix self.). Under no circumstance can you introduce new input variables.\n'}]
      ''')


      messages:  [{'role': 'system', 'content': 'You are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effective as possible.
Your goal is to write a reward function for the environment that will help the agent learn the task described in text. 
Your reward function should use useful variables from the environment as inputs. As an example,
the reward function signature can be: @torch.jit.script
def compute_reward(object_pos: torch.Tensor, goal_pos: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:
    ...
    return reward, {}

Since the reward function will be decorated with @torch.jit.script,
please make sure that the code is compatible with TorchScript (e.g., use torch tensor instead of numpy array). 
Make sure any new tensor or variable you introduce is on the same device as the input tensors. The output of the reward function should consist of two items:
    (1) the total reward,
    (2) a dictionary of each individual re